# variance in denial rate via random sampling

```
Starting...
Done. Compiling results.                

Trials Run: 500000

Results for: n denying:
=======================
- Mean(statistic=660.001498, minmax=(659.9646687985933, 660.0383272014067))
- Variance(statistic=250.66869175599598, minmax=(249.84406514239993, 251.49331836959203))
- Std_dev(statistic=15.832520069653977, minmax=(15.806477891593625, 15.85856224771433))

Results for: P(deny):
=====================
- Mean(statistic=0.4000009078787878, minmax=(0.39997858715066253, 0.40002322860691303))
- Variance(statistic=9.207298136124737e-05, minmax=(9.177008820657481e-05, 9.237587451591993e-05))
- Std_dev(statistic=0.009595466708881199, minmax=(0.009579683570662804, 0.009611249847099594))
```

```
Trials Run: 50000

Denying members | Min: 594, Max: 727
P(deny)         | Min: 0.36, Max: 0.4406060606060606

Results for: n denying:
=======================
- Mean(statistic=660.01426, minmax=(659.8978372145845, 660.1306827854156))
- Variance(statistic=250.49061665239998, minmax=(247.88477084380597, 253.096462460994))
- Std_dev(statistic=15.82689535734662, minmax=(15.74457201629463, 15.90921869839861))

Results for: P(deny):
=====================
- Mean(statistic=0.40000864242424233, minmax=(0.3999380831603541, 0.40007920168813055))
- Variance(statistic=9.200757269142332e-05, minmax=(9.105042087926759e-05, 9.296472450357906e-05))
- Std_dev(statistic=0.009592057792331285, minmax=(0.009542164858360382, 0.009641950726302188))
```

```
Trials Run: 500000

Denying members | Min: 585, Max: 740
P(deny)         | Min: 0.35454545454545455, Max: 0.4484848484848485

Results for: n denying:
=======================
- Mean(statistic=660.029864, minmax=(659.9930150989986, 660.0667129010013))
- Variance(statistic=250.936924141504, minmax=(250.1114151218836, 251.7624331611244))
- Std_dev(statistic=15.840988736234364, minmax=(15.81493262845707, 15.867044844011657))

Results for: P(deny):
=====================
- Mean(statistic=0.40001809939393945, minmax=(0.3999957667266659, 0.400040432061213))
- Variance(statistic=9.217150565344503e-05, minmax=(9.186828838269374e-05, 9.247472292419632e-05))
- Std_dev(statistic=0.009600599234081434, minmax=(0.009584807653610347, 0.009616390814552522))
```

In [5]:
import math
import numpy as np
import scipy.stats as stats

rand = np.random.Generator(np.random.PCG64DXSM())

def pad_and_print(s, pad=0):
    to_print = s + " " * max(0, pad-len(s))
    print(to_print, end='')
    return len(to_print)

def measure_variance(repeat_times=10_000, n_members=4500, sample_size=1650, overall_denial_rate=0.4):
    p_deny = overall_denial_rate
    
    n_deny = round(p_deny * n_members)
    n_conf = n_members - n_deny

    # since we want to count the number of denying members in samples, 
    # using `1` for denying members and `0` for confirming members means
    # that we can just `sum` the sample.
    members = [1] * n_deny + [0] * n_conf

    results = []
    results_p = []

    print(f"Starting...")
    progress_mod = repeat_times // 20
    max_len = 11
    for t in range(repeat_times):
        member_list = rand.choice(members, sample_size, replace=False, shuffle=True)
        measured_denying = sum(member_list)
        measured_p_deny = measured_denying / sample_size
        results.append(measured_denying)
        results_p.append(measured_p_deny)
        if t % progress_mod == 0:
            status_msg = f"\r{t} / {repeat_times}: {measured_denying} denying (P = {measured_p_deny:.3f})"
            max_len = pad_and_print(status_msg, pad=max_len)
    
    pad_and_print(f"\rDone. Compiling results.", pad=max_len)
    print()
    print(f"Trials Run: {repeat_times}")
    print()
    print(f"Denying members | Min: {min(results)}, Max: {max(results)}")
    print(f"P(deny)         | Min: {min(results_p)}, Max: {max(results_p)}")
    print()
    
    for name, res in [('n denying', results), ('P(deny)', results_p)]:
        mvs = stats.bayes_mvs(res)
        res_title = f"Results for: {name}:"
        print(res_title)
        print("=" * len(res_title))
        for smm in mvs:
            print(f"- {smm}")
        print()

measure_variance(repeat_times=500_000)

Starting...
Done. Compiling results.                
Trials Run: 500000

Denying members | Min: 585, Max: 740
P(deny)         | Min: 0.35454545454545455, Max: 0.4484848484848485

Results for: n denying:
- Mean(statistic=660.029864, minmax=(659.9930150989986, 660.0667129010013))
- Variance(statistic=250.936924141504, minmax=(250.1114151218836, 251.7624331611244))
- Std_dev(statistic=15.840988736234364, minmax=(15.81493262845707, 15.867044844011657))

Results for: P(deny):
- Mean(statistic=0.40001809939393945, minmax=(0.3999957667266659, 0.400040432061213))
- Variance(statistic=9.217150565344503e-05, minmax=(9.186828838269374e-05, 9.247472292419632e-05))
- Std_dev(statistic=0.009600599234081434, minmax=(0.009584807653610347, 0.009616390814552522))

